In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics

from warnings import simplefilter

import joblib

2024-02-26 20:26:36.859323: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 20:26:36.862010: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 20:26:36.891197: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 20:26:36.891251: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 20:26:36.892239: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test.reshape(-1), y_pred_classes, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
x_train = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/x_train.csv', low_memory=False)
y_train = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/y_train.csv', low_memory=False)
x_test = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/x_test.csv', low_memory=False)
y_test = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/y_test.csv', low_memory=False)

x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

In [4]:
model = joblib.load("/home/jovyan/UNSW/dl/new_dl/dnn.joblib")

2024-02-26 20:26:42.404107: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://2d4f21190b514a8e984134c1581c06ee: INVALID_ARGUMENT: ram://2d4f21190b514a8e984134c1581c06ee is a directory.


In [5]:
np.unique(y_train)

array([0, 1])

In [6]:
x_train.shape

(82332, 56)

In [7]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=2,
    input_shape=(56,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [8]:
attack = FastGradientMethod(estimator=classifier, eps=0.01)

x_test_adv = attack.generate(x=x_test)

In [9]:
calculate_performance_metrics(x_test_adv, y_test, model)

5480/5480 [==============================] - 3s 566us/step
Accuracy: 0.9041296673339378
AUC: 0.9135683104806515

macro
Precision: 0.8828706936921161
Recall: 0.9135683104806515
F1 Score: 0.8943750682772997

weighted
Precision: 0.9140202610993343
Recall: 0.9041296673339378
F1 Score: 0.9059705652998487

Mean FNR: 0.08643168951934851
Mean TNR: 0.9135683104806515
Mean FPR: 0.08643168951934851
Mean TPR: 0.9135683104806515


In [10]:
attack = FastGradientMethod(estimator=classifier, eps=0.05)

x_test_adv = attack.generate(x=x_test)

In [11]:
calculate_performance_metrics(x_test_adv, y_test, model)

5480/5480 [==============================] - 3s 609us/step
Accuracy: 0.8894554040412681
AUC: 0.8933720895225806

macro
Precision: 0.8673554042125237
Recall: 0.8933720895225805
F1 Score: 0.8775439249285664

weighted
Precision: 0.897766129357589
Recall: 0.8894554040412681
F1 Score: 0.8913406004666867

Mean FNR: 0.10662791047741944
Mean TNR: 0.8933720895225805
Mean FPR: 0.10662791047741944
Mean TPR: 0.8933720895225805


In [14]:
attack = FastGradientMethod(estimator=classifier, eps=0.1)

x_test_adv = attack.generate(x=x_test)

In [17]:
calculate_performance_metrics(x_test_adv, y_test, model)

5480/5480 [==============================] - 3s 561us/step
Accuracy: 0.8644698045522724
AUC: 0.8675770560530629

macro
Precision: 0.8406207620386796
Recall: 0.8675770560530629
F1 Score: 0.8505884666013921

weighted
Precision: 0.8753086355111701
Recall: 0.8644698045522724
F1 Score: 0.867040105241989

Mean FNR: 0.13242294394693715
Mean TNR: 0.8675770560530629
Mean FPR: 0.13242294394693715
Mean TPR: 0.8675770560530629


In [18]:
attack = FastGradientMethod(estimator=classifier, eps=0.15)

x_test_adv = attack.generate(x=x_test)

In [19]:
calculate_performance_metrics(x_test_adv, y_test, model)

5480/5480 [==============================] - 3s 567us/step
Accuracy: 0.8198310720253678
AUC: 0.8295669450955067

macro
Precision: 0.7966243116102427
Recall: 0.8295669450955065
F1 Score: 0.8053533901506302

weighted
Precision: 0.8421346807069464
Recall: 0.8198310720253678
F1 Score: 0.8245301121838327

Mean FNR: 0.17043305490449337
Mean TNR: 0.8295669450955065
Mean FPR: 0.17043305490449337
Mean TPR: 0.8295669450955065


In [20]:
attack = FastGradientMethod(estimator=classifier, eps=0.2)

x_test_adv = attack.generate(x=x_test)

In [21]:
calculate_performance_metrics(x_test_adv, y_test, model)

5480/5480 [==============================] - 3s 562us/step
Accuracy: 0.7524823059067759
AUC: 0.7767643681162143

macro
Precision: 0.7416848940649124
Recall: 0.7767643681162144
F1 Score: 0.740674484072503

weighted
Precision: 0.8012130395497652
Recall: 0.7524823059067759
F1 Score: 0.7606643074803523

Mean FNR: 0.2232356318837856
Mean TNR: 0.7767643681162144
Mean FPR: 0.2232356318837856
Mean TPR: 0.7767643681162144


In [22]:
attack = FastGradientMethod(estimator=classifier, eps=0.25)

x_test_adv = attack.generate(x=x_test)

In [23]:
calculate_performance_metrics(x_test_adv, y_test, model)

5480/5480 [==============================] - 3s 566us/step
Accuracy: 0.7071192704501514
AUC: 0.7416152183060066

macro
Precision: 0.7101416406311478
Recall: 0.7416152183060067
F1 Score: 0.6981463354652429

weighted
Precision: 0.7766369340088798
Recall: 0.7071192704501514
F1 Score: 0.7169467282699498

Mean FNR: 0.2583847816939933
Mean TNR: 0.7416152183060067
Mean FPR: 0.2583847816939933
Mean TPR: 0.7416152183060067


In [24]:
attack = FastGradientMethod(estimator=classifier, eps=0.3)

x_test_adv = attack.generate(x=x_test)

In [25]:
calculate_performance_metrics(x_test_adv, y_test, model)

5480/5480 [==============================] - 3s 611us/step
Accuracy: 0.6676133933307099
AUC: 0.7108589037176781

macro
Precision: 0.6847432513508578
Recall: 0.7108589037176781
F1 Score: 0.6612477900383695

weighted
Precision: 0.7558037226884824
Recall: 0.6676133933307099
F1 Score: 0.6780227808080536

Mean FNR: 0.2891410962823219
Mean TNR: 0.7108589037176781
Mean FPR: 0.2891410962823219
Mean TPR: 0.7108589037176781
